# pip libarary

In [ ]:
from io import StringIO
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import ast
import time
import copy
import os
import json
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# CVRPTWSolver

In [ ]:
class CVRPTWSolver:
    def __init__(self, distance_matrix, vehicle_number, vehicle_capacity, 
                 demands, time_windows, service_times, initial_solution=None,
                 time_window_penalty=1000, capacity_penalty=1000):
        self.distance_matrix = distance_matrix
        self.vehicle_number = vehicle_number
        self.vehicle_capacity = vehicle_capacity
        self.demands = demands
        self.time_windows = time_windows
        self.service_times = service_times
        self.time_window_penalty = time_window_penalty
        self.capacity_penalty = capacity_penalty
        self.num_customers = len(demands) - 1 
        
        if initial_solution is None:
            self.current_solution = self._create_initial_solution()
            print(f"create casual initial solution is {self.current_solution}")
        else:
            self.current_solution = copy.deepcopy(initial_solution)
        
        self.best_solution = copy.deepcopy(self.current_solution)
        self.best_cost = self._calculate_total_cost(self.best_solution)
        
    def _create_initial_solution(self):
        solution = []
        unassigned = list(range(1, self.num_customers + 1))
        
        for v in range(self.vehicle_number):
            if not unassigned:
                break
                
            route = [0]
            current_capacity = 0
            current_time = 0
            
            while unassigned:
                best_customer = None
                best_insertion_cost = float('inf')
                
                for customer in unassigned:
                    if current_capacity + self.demands[customer] > self.vehicle_capacity:
                        continue
                        
                    travel_time = self.distance_matrix[route[-1]][customer]
                    arrival_time = current_time + travel_time
                    
                    earliest, latest = self.time_windows[customer]
                    wait_time = max(0, earliest - arrival_time)
                    delay_time = max(0, arrival_time - latest)
                    
                    if delay_time > 0:
                        continue
                        
                    insertion_cost = travel_time + wait_time
                    
                    if insertion_cost < best_insertion_cost:
                        best_insertion_cost = insertion_cost
                        best_customer = customer
                
                if best_customer is None:
                    break
                    
                route.append(best_customer)
                unassigned.remove(best_customer)
                current_capacity += self.demands[best_customer]
                
                travel_time = self.distance_matrix[route[-2]][best_customer]
                arrival_time = current_time + travel_time
                earliest, latest = self.time_windows[best_customer]
                service_start = max(arrival_time, earliest)
                current_time = service_start + self.service_times[best_customer]
            
            route.append(0)
            solution.append(route)

        while unassigned:
            route = [0]
            customers_in_route = []
            current_capacity = 0
            
            for customer in sorted(unassigned, key=lambda x: self.time_windows[x][0]):
                if current_capacity + self.demands[customer] <= self.vehicle_capacity:
                    route.append(customer)
                    customers_in_route.append(customer)
                    current_capacity += self.demands[customer]
                    
            for customer in customers_in_route:
                unassigned.remove(customer)
                
            route.append(0)
            solution.append(route)
            
        return solution
    
    def _is_feasible_route(self, route):
        if not route or len(route) <= 2:
            return True, 0, 0
            
        total_demand = sum(self.demands[customer] for customer in route[1:-1])
        capacity_violation = max(0, total_demand - self.vehicle_capacity)
        
        current_time = 0
        time_window_violation = 0
        
        for i in range(1, len(route)):
            prev_node = route[i-1]
            curr_node = route[i]
            
            travel_time = self.distance_matrix[prev_node][curr_node]
            arrival_time = current_time + travel_time
            
            if curr_node != 0: 
                earliest, latest = self.time_windows[curr_node]
                
                service_start = max(arrival_time, earliest)
                delay = max(0, arrival_time - latest)
                time_window_violation += delay

                current_time = service_start + self.service_times[curr_node]
            else:
                current_time = arrival_time
        
        is_feasible = (capacity_violation == 0) and (time_window_violation == 0)
        return is_feasible, capacity_violation, time_window_violation
    
    def _calculate_route_cost(self, route):
        # if not route or len(route) <= 2:
        #     return 0
            
        if not isinstance(route, list) or len(route) <= 2:
            return 0  # Invalid or empty route

        # distance_cost = sum(self.distance_matrix[route[i-1]][route[i]] for i in range(1, len(route)))

        # Flatten the route if it's a list of lists
        if isinstance(route[0], list):
            route = [customer for subroute in route for customer in subroute]

        # Now calculate the distance cost assuming route is a flat list
        distance_cost = sum(self.distance_matrix[route[i-1]][route[i]] for i in range(1, len(route)))

        _, capacity_violation, time_window_violation = self._is_feasible_route(route)
        
        total_cost = distance_cost
        total_cost += self.capacity_penalty * capacity_violation
        total_cost += self.time_window_penalty * time_window_violation
        
        return total_cost
    
    def _calculate_total_cost(self, solution):
        return sum(self._calculate_route_cost(route) for route in solution)
    
    def _remove_customer(self, customer, solution):
        for i, route in enumerate(solution):
            if customer in route:
                route_index = i
                position = route.index(customer)
                route.pop(position)
                return solution, route_index
        return solution, -1
    
    def _best_insertion_position(self, customer, route):
        best_position = 0
        best_cost_change = float('inf')
        original_cost = self._calculate_route_cost(route)
        
        for i in range(1, len(route)):
            new_route = route.copy()
            new_route.insert(i, customer)
            new_cost = self._calculate_route_cost(new_route)
            cost_change = new_cost - original_cost
            
            if cost_change < best_cost_change:
                best_cost_change = cost_change
                best_position = i
                
        return best_position, best_cost_change
    
    def _relocate_operator(self, solution):
        new_solution = copy.deepcopy(solution)
        
        non_empty_routes = [i for i, route in enumerate(new_solution) if len(route) > 2]
        if not non_empty_routes:
            return new_solution
            
        source_route_idx = random.choice(non_empty_routes)
        source_route = new_solution[source_route_idx]
        
        if len(source_route) <= 2:
            return new_solution
            
        customer_positions = list(range(1, len(source_route) - 1))
        if not customer_positions:
            return new_solution
            
        customer_pos = random.choice(customer_positions)
        customer = source_route[customer_pos]
        
        source_route.pop(customer_pos)
        
        best_route_idx = -1
        best_position = -1
        best_cost_change = float('inf')
        
        for route_idx, route in enumerate(new_solution):
            if len(route) < 2:
                continue
                
            position, cost_change = self._best_insertion_position(customer, route)
            
            if cost_change < best_cost_change:
                best_cost_change = cost_change
                best_position = position
                best_route_idx = route_idx
        
        if best_route_idx != -1:
            new_solution[best_route_idx].insert(best_position, customer)
        else:
            source_route.insert(customer_pos, customer)
            
        return new_solution
    
    def _exchange_operator(self, solution):
        new_solution = copy.deepcopy(solution)
        
        non_empty_routes = [i for i, route in enumerate(new_solution) if len(route) > 2]
        if len(non_empty_routes) < 1:
            return new_solution
            
        route1_idx = random.choice(non_empty_routes)
        route1 = new_solution[route1_idx]
        
        if len(non_empty_routes) > 1 and random.random() < 0.5:
            remaining_routes = [i for i in non_empty_routes if i != route1_idx]
            route2_idx = random.choice(remaining_routes)
        else:
            route2_idx = route1_idx
            
        route2 = new_solution[route2_idx]
        
        customers1 = list(range(1, len(route1) - 1))
        if not customers1:
            return new_solution
            
        pos1 = random.choice(customers1)
        
        if route1_idx == route2_idx:
            remaining_positions = [i for i in range(1, len(route1) - 1) if i != pos1]
            if not remaining_positions:
                return new_solution
            pos2 = random.choice(remaining_positions)
        else:
            customers2 = list(range(1, len(route2) - 1))
            if not customers2:
                return new_solution
            pos2 = random.choice(customers2)
        
        customer1 = route1[pos1]
        customer2 = route2[pos2]
        
        route1[pos1] = customer2
        route2[pos2] = customer1
        
        return new_solution
    
    def _two_opt_operator(self, solution):
        new_solution = copy.deepcopy(solution)
        
        non_empty_routes = [i for i, route in enumerate(new_solution) if len(route) > 3]
        if not non_empty_routes:
            return new_solution
            
        route_idx = random.choice(non_empty_routes)
        route = new_solution[route_idx]
        
        positions = list(range(1, len(route) - 1))
        if len(positions) < 2:
            return new_solution
            
        pos1 = random.choice(positions)
        remaining_pos = [p for p in positions if p != pos1]
        pos2 = random.choice(remaining_pos)
        
        if pos1 > pos2:
            pos1, pos2 = pos2, pos1
            
        new_solution[route_idx][pos1:pos2+1] = reversed(route[pos1:pos2+1])
        
        return new_solution
    
    def _or_opt_operator(self, solution):
        new_solution = copy.deepcopy(solution)
        
        non_empty_routes = [i for i, route in enumerate(new_solution) if len(route) > 3]
        if not non_empty_routes:
            return new_solution
            
        route_idx = random.choice(non_empty_routes)
        route = new_solution[route_idx]
        
        seq_length = min(random.randint(1, 3), len(route) - 3)
        if seq_length <= 0:
            return new_solution
            
        max_start = len(route) - 1 - seq_length
        if max_start < 1:
            return new_solution
            
        start_pos = random.randint(1, max_start)
    
        sequence = route[start_pos:start_pos + seq_length]
        
        for _ in range(seq_length):
            route.pop(start_pos)
        
        possible_positions = [i for i in range(1, len(route)) if i != start_pos]
        if not possible_positions:
            for i, customer in enumerate(sequence):
                route.insert(start_pos + i, customer)
            return new_solution
            
        new_pos = random.choice(possible_positions)
        
        for i, customer in enumerate(sequence):
            route.insert(new_pos + i, customer)
            
        return new_solution
    
    def _route_fusion_operator(self, solution):

        new_solution = copy.deepcopy(solution)
        
        non_empty_routes = [i for i, route in enumerate(new_solution) if len(route) > 2]
        if len(non_empty_routes) < 2:
            return new_solution
            
        route1_idx = random.choice(non_empty_routes)
        remaining_routes = [i for i in non_empty_routes if i != route1_idx]
        route2_idx = random.choice(remaining_routes)
        
        route1 = new_solution[route1_idx]
        route2 = new_solution[route2_idx]

        merged_route = [0] + route1[1:-1] + route2[1:-1] + [0]
        
        is_feasible, _, _ = self._is_feasible_route(merged_route)
        
        if is_feasible:
            new_solution[route1_idx] = merged_route
            new_solution.pop(route2_idx)
        
        return new_solution
    
    def _adaptive_large_neighborhood_search(self, current_solution, num_destroy=3, num_repair=2):
        solution = copy.deepcopy(current_solution)
        
        all_customers = []
        for route in solution:
            all_customers.extend([customer for customer in route[1:-1]])
            
        if not all_customers:
            return solution
        
        num_to_destroy = min(num_destroy, len(all_customers))
        to_remove = random.sample(all_customers, num_to_destroy)
        
        removed_customers = []
        for customer in to_remove:
            for i, route in enumerate(solution):
                if customer in route:
                    pos = route.index(customer)
                    route.pop(pos)
                    removed_customers.append(customer)
                    break
        
        for _ in range(num_repair):
            if not removed_customers:
                break
                
            customer = removed_customers.pop(random.randint(0, len(removed_customers) - 1))
            
            best_route_idx = -1
            best_position = -1
            best_cost_change = float('inf')
            
            for route_idx, route in enumerate(solution):
                if len(route) < 2:
                    continue
                    
                position, cost_change = self._best_insertion_position(customer, route)
                
                if cost_change < best_cost_change:
                    best_cost_change = cost_change
                    best_position = position
                    best_route_idx = route_idx
            
            if best_route_idx != -1:
                solution[best_route_idx].insert(best_position, customer)
            else:
                solution.append([0, customer, 0])
        
        for customer in removed_customers:
            solution.append([0, customer, 0])
            
        solution = [route for route in solution if len(route) > 2]
            
        return solution
    
    def simulated_annealing(self, initial_temp=1000, alpha=0.98, iterations_per_temp=100, min_temp=0.1, max_iterations=100000):
        current_solution = copy.deepcopy(self.current_solution)
        current_cost = self._calculate_total_cost(current_solution)
        
        best_solution = copy.deepcopy(current_solution)
        best_cost = current_cost
        
        temperature = initial_temp
        iteration = 0
        no_improvement = 0
        
        operators = [
            self._relocate_operator,
            self._exchange_operator,
            self._two_opt_operator,
            self._or_opt_operator,
            self._route_fusion_operator,
            self._adaptive_large_neighborhood_search
        ]
        
        weights = [1.0] * len(operators)
        
        success_count = [0] * len(operators)
        total_count = [0] * len(operators)
        
        while temperature > min_temp and iteration < max_iterations:
            for _ in range(iterations_per_temp):
                op_idx = random.choices(range(len(operators)), weights=weights, k=1)[0]
                operator = operators[op_idx]

                new_solution = operator(current_solution)
                new_cost = self._calculate_total_cost(new_solution)

                delta = new_cost - current_cost

                total_count[op_idx] += 1
                
                if delta < 0 or random.random() < np.exp(-delta / temperature):
                    current_solution = new_solution
                    current_cost = new_cost

                    success_count[op_idx] += 1

                    if current_cost < best_cost:
                        best_solution = copy.deepcopy(current_solution)
                        best_cost = current_cost
                        no_improvement = 0
                    else:
                        no_improvement += 1
                else:
                    no_improvement += 1
                
                iteration += 1

            for i in range(len(weights)):
                if total_count[i] > 0:
                    weights[i] = 0.5 * (success_count[i] / max(1, total_count[i])) + 0.5

            success_count = [0] * len(operators)
            total_count = [0] * len(operators)

            temperature *= alpha

            if no_improvement > 1000:
                current_solution = self._adaptive_large_neighborhood_search(best_solution, 
                                                                           num_destroy=min(10, len(best_solution)),
                                                                           num_repair=8)
                current_cost = self._calculate_total_cost(current_solution)
                no_improvement = 0
        
        return best_solution, best_cost
    
    def variable_neighborhood_search(self, max_iterations=1000, max_no_improvement=100):
        current_solution = copy.deepcopy(self.current_solution)
        current_cost = self._calculate_total_cost(current_solution)
        
        best_solution = copy.deepcopy(current_solution)
        best_cost = current_cost

        neighborhoods = [
            self._relocate_operator,
            self._exchange_operator,
            self._two_opt_operator,
            self._or_opt_operator,
            self._route_fusion_operator,
            lambda x: self._adaptive_large_neighborhood_search(x, num_destroy=2, num_repair=2),
            lambda x: self._adaptive_large_neighborhood_search(x, num_destroy=4, num_repair=3)
        ]
        
        iteration = 0
        no_improvement = 0
        k = 0
        
        while iteration < max_iterations and no_improvement < max_no_improvement:
            local_best_solution = copy.deepcopy(current_solution)
            local_best_cost = current_cost
            
            for _ in range(10):
                new_solution = neighborhoods[k](current_solution)
                new_cost = self._calculate_total_cost(new_solution)
                
                if new_cost < local_best_cost:
                    local_best_solution = new_solution
                    local_best_cost = new_cost

            if local_best_cost < current_cost:
                current_solution = local_best_solution
                current_cost = local_best_cost
                k = 0 

                if current_cost < best_cost:
                    best_solution = copy.deepcopy(current_solution)
                    best_cost = current_cost
                    no_improvement = 0
                else:
                    no_improvement += 1
            else:
                k = (k + 1) % len(neighborhoods)
                no_improvement += 1
            
            iteration += 1

            if k == 0 and no_improvement > 50:
                current_solution = self._adaptive_large_neighborhood_search(best_solution, 
                                                                           num_destroy=5, 
                                                                           num_repair=4)
                current_cost = self._calculate_total_cost(current_solution)
                no_improvement = 0
        
        return best_solution, best_cost
    
    def simulated_annealing_time_limited(
        self,
        time_limit=300,
        initial_temp=1000,
        alpha=0.98,
        iterations_per_temp=100,
        known_optimum=None
    ):
        import time
        start_time = time.time()

        current_solution = copy.deepcopy(self.current_solution)
        current_cost = self._calculate_total_cost(current_solution)
        best_solution = copy.deepcopy(current_solution)
        best_cost = current_cost

        temperature = initial_temp
        no_improvement = 0

        while True:
            if time.time() - start_time >= time_limit:
                break  

            for _ in range(iterations_per_temp):
                if time.time() - start_time >= time_limit:
                    break

                operators = [
                    self._relocate_operator,
                    self._exchange_operator,
                    self._two_opt_operator,
                    self._or_opt_operator,
                    self._route_fusion_operator,
                    self._adaptive_large_neighborhood_search
                ]
                operator = random.choice(operators)
                
                new_solution = operator(current_solution)
                new_cost = self._calculate_total_cost(new_solution)
                
                delta = new_cost - current_cost
                if delta < 0 or random.random() < np.exp(-delta / temperature):
                    current_solution = new_solution
                    current_cost = new_cost
                    if current_cost < best_cost:
                        best_solution = copy.deepcopy(current_solution)
                        best_cost = current_cost
                        no_improvement = 0
                    else:
                        no_improvement += 1
                else:
                    no_improvement += 1

                if known_optimum is not None and best_cost <= known_optimum:
                    return best_solution, best_cost

            temperature *= alpha
            if temperature < 0.1:
                temperature = 0.1
            
            if no_improvement > 1000:
                current_solution = self._adaptive_large_neighborhood_search(
                    best_solution,
                    num_destroy=10,
                    num_repair=8
                )
                current_cost = self._calculate_total_cost(current_solution)
                no_improvement = 0
        
        return best_solution, best_cost


    def variable_neighborhood_search_time_limited(
        self,
        time_limit=300,
        known_optimum=None
    ):
        import time
        start_time = time.time()

        current_solution = copy.deepcopy(self.current_solution)
        current_cost = self._calculate_total_cost(current_solution)
        best_solution = copy.deepcopy(current_solution)
        best_cost = current_cost

        neighborhoods = [
            self._relocate_operator,
            self._exchange_operator,
            self._two_opt_operator,
            self._or_opt_operator,
            self._route_fusion_operator,
            lambda x: self._adaptive_large_neighborhood_search(x, num_destroy=2, num_repair=2),
            lambda x: self._adaptive_large_neighborhood_search(x, num_destroy=4, num_repair=3)
        ]
        k = 0
        no_improvement = 0

        while True:
            if time.time() - start_time >= time_limit:
                break
            
            local_best_solution = copy.deepcopy(current_solution)
            local_best_cost = current_cost
            
            for _ in range(10):
                if time.time() - start_time >= time_limit:
                    break
                
                new_solution = neighborhoods[k](current_solution)
                new_cost = self._calculate_total_cost(new_solution)
                if new_cost < local_best_cost:
                    local_best_solution = new_solution
                    local_best_cost = new_cost
            
            if local_best_cost < current_cost:
                current_solution = local_best_solution
                current_cost = local_best_cost
                if current_cost < best_cost:
                    best_solution = copy.deepcopy(current_solution)
                    best_cost = current_cost
                    no_improvement = 0
                else:
                    no_improvement += 1
                k = 0
            else:
                k = (k + 1) % len(neighborhoods)
                no_improvement += 1

            if known_optimum is not None and best_cost <= known_optimum:
                return best_solution, best_cost
            
            if k == 0 and no_improvement > 50:
                current_solution = self._adaptive_large_neighborhood_search(
                    best_solution,
                    num_destroy=5,
                    num_repair=4
                )
                current_cost = self._calculate_total_cost(current_solution)
                no_improvement = 0

        return best_solution, best_cost


    
    def hybrid_metaheuristic(self, time_limit=300):
        start_time = time.time()

        sa_solution, sa_cost = self.simulated_annealing(
            initial_temp=100,
            alpha=0.98,
            iterations_per_temp=50,
            min_temp=1.0,
            max_iterations=1000
        )

        if sa_cost < self._calculate_total_cost(self.best_solution):
            self.best_solution = copy.deepcopy(sa_solution)
            self.best_cost = sa_cost

        if time.time() - start_time >= time_limit:
            return self.best_solution, self.best_cost

        self.current_solution = copy.deepcopy(self.best_solution)
        vns_solution, vns_cost = self.variable_neighborhood_search(
            max_iterations=2000,
            max_no_improvement=200
        )

        if vns_cost < self.best_cost:
            self.best_solution = copy.deepcopy(vns_solution)
            self.best_cost = vns_cost

        if time.time() - start_time >= time_limit:
            return self.best_solution, self.best_cost

        remaining_time = time_limit - (time.time() - start_time)
        iterations = min(500, int(remaining_time / 0.1))
        
        current_solution = copy.deepcopy(self.best_solution)
        current_cost = self.best_cost
        
        for _ in range(iterations):
            if time.time() - start_time >= time_limit:
                break

            destroy_size = random.randint(3, 8)
            repair_size = random.randint(2, destroy_size)
            
            new_solution = self._adaptive_large_neighborhood_search(
                current_solution,
                num_destroy=destroy_size,
                num_repair=repair_size
            )
            new_cost = self._calculate_total_cost(new_solution)
            
            if new_cost < current_cost:
                current_solution = new_solution
                current_cost = new_cost
                
                if new_cost < self.best_cost:
                    self.best_solution = copy.deepcopy(new_solution)
                    self.best_cost = new_cost
        
        return self.best_solution, self.best_cost

    # def solve(self, time_limit=300, method="hybrid"):
    #     start_time = time.time()
        
    #     if method == "sa":
    #         self.best_solution, self.best_cost = self.simulated_annealing(
    #             initial_temp=100,
    #             alpha=0.98,
    #             iterations_per_temp=100,
    #             min_temp=0.1,
    #             max_iterations=10000
    #         )
    #     elif method == "vns":
    #         self.best_solution, self.best_cost = self.variable_neighborhood_search(
    #             max_iterations=5000,
    #             max_no_improvement=500
    #         )
    #     else:  # hybrid method
    #         self.best_solution, self.best_cost = self.hybrid_metaheuristic(time_limit)
        
    #     end_time = time.time()
    #     solution_time = end_time - start_time
        
    #     solution_info = self._analyze_solution(self.best_solution)
    #     solution_info["total_cost"] = self.best_cost
    #     solution_info["solution_time"] = solution_time
    #     solution_info["method"] = method
        
    #     print(f"Solution completed, total cost: {self.best_cost:.2f}, time: {solution_time:.2f} seconds")
    #     return self.best_solution, self.best_cost, solution_info

    def solve(self, time_limit=300, method="vns", known_optimum=828.94):
        start_time = time.time()

        self.best_solution = copy.deepcopy(self.current_solution)
        self.best_cost = self._calculate_total_cost(self.best_solution)

        if method == "sa":
            best_solution, best_cost = self.simulated_annealing(
                initial_temp=1000,
                alpha=0.98,
                iterations_per_temp=100,
                min_temp=0.1,
                max_iterations=100000
            )
        elif method == "satl":
            best_solution, best_cost = self.simulated_annealing_time_limited(
                time_limit=time_limit,
                known_optimum=known_optimum
            )
        elif method == "vns":
            best_solution, best_cost = self.variable_neighborhood_search(
                max_iterations=1000,
                max_no_improvement=100
            )
        elif method == "vnstl":
            best_solution, best_cost = self.variable_neighborhood_search_time_limited(
                time_limit=time_limit
            )
        elif method == "hybrid":
            best_solution, best_cost = self.hybrid_metaheuristic(time_limit=time_limit)
        else:
            raise ValueError(f"unknow method：{method}")

        end_time = time.time()
        solution_time = end_time - start_time

        if best_cost < self.best_cost:
            self.best_solution = copy.deepcopy(best_solution)
            self.best_cost = best_cost

        # 输出或分析
        solution_info = self._analyze_solution(self.best_solution)
        solution_info["total_cost"] = self.best_cost
        solution_info["solution_time"] = solution_time
        solution_info["method"] = method

        print(f"Solution completed, total cost: {self.best_cost:.2f}, time: {solution_time:.2f} seconds")

        return self.best_solution, self.best_cost, solution_info


    def _analyze_solution(self, solution):
        info = {
            "num_routes": len(solution),
            "total_distance": 0,
            "total_time_window_violations": 0,
            "total_capacity_violations": 0,
            "route_info": []
        }
        
        for i, route in enumerate(solution):
            route_distance = sum(self.distance_matrix[route[j-1]][route[j]] for j in range(1, len(route)))
            _, capacity_violation, time_window_violation = self._is_feasible_route(route)
            
            info["total_distance"] += route_distance
            info["total_time_window_violations"] += time_window_violation
            info["total_capacity_violations"] += capacity_violation
            
            route_info = {
                "route_id": i,
                "route": route,
                "distance": route_distance,
                "customers": len(route) - 2, 
                "total_demand": sum(self.demands[customer] for customer in route[1:-1]),
                "capacity_violation": capacity_violation,
                "time_window_violation": time_window_violation,
                "is_feasible": (capacity_violation == 0 and time_window_violation == 0)
            }
            
            info["route_info"].append(route_info)
        
        if info["num_routes"] > 0:
            info["avg_customers_per_route"] = sum(r["customers"] for r in info["route_info"]) / info["num_routes"]
            info["avg_distance_per_route"] = info["total_distance"] / info["num_routes"]
        else:
            info["avg_customers_per_route"] = 0
            info["avg_distance_per_route"] = 0
            
        return info
    
    def visualize_solution(self, solution=None, coordinates=None, save_path=None):
        if solution is None:
            solution = self.best_solution
            
        if coordinates is None:
            print("No coordinates")
            return
            
        try:
            plt.figure(figsize=(12, 8))
            
            colors = list(mcolors.TABLEAU_COLORS.values())
            
            for node, (x, y) in coordinates.items():
                if node == 0:  
                    plt.scatter(x, y, c='red', s=100, marker='*', label='Depot')
                else:  
                    plt.scatter(x, y, c='blue', s=30)
                    plt.text(x, y, str(node), fontsize=8)
            
            for i, route in enumerate(solution):
                route_color = colors[i % len(colors)]
                route_x = [coordinates[node][0] for node in route]
                route_y = [coordinates[node][1] for node in route]
                
                plt.plot(route_x, route_y, c=route_color, linewidth=1.5, label=f'Route {i+1}')
            
            plt.title('CVRPTW Solution Visualization')
            plt.xlabel('X Coordinate')
            plt.ylabel('Y Coordinate')
            plt.legend(loc='best')
            plt.grid(True, linestyle='--', alpha=0.7)
            
            if save_path:
                plt.savefig(save_path, dpi=300, bbox_inches='tight')
                print(f"Image saved to {save_path}")
            else:
                plt.show()
                
        except ImportError:
            print("No matplotlib")

# produce data

In [ ]:
def produce_train_data(file_path, time_limit=10, method="satl", known_optimum=None, initial_solution=None, pic_save_path=None, base_name=None): 
    def load_solomon_data(f_path):
        try:
            with open(f_path, 'r') as file:
                lines = file.readlines()
        except FileNotFoundError:
            print(f"Error: File not found at {f_path}")
            return None

        vehicle_number = 0
        vehicle_capacity = 0
        demands = []
        time_windows = []
        coords = []
        service_times = []
        coordinates_dict = {}

        try:
            vehicle_header_index = -1
            for i, line in enumerate(lines):
                 if "VEHICLE" in line.upper():
                      vehicle_header_index = i
                      break

            if vehicle_header_index == -1 or vehicle_header_index + 2 >= len(lines):
                 raise ValueError("Vehicle information header not found or incomplete.")

            vehicle_info_line = lines[vehicle_header_index + 2]
            vehicle_number_line = vehicle_info_line.split()
            if len(vehicle_number_line) < 2:
                 raise ValueError("Vehicle number/capacity line format error.")
            vehicle_number = int(vehicle_number_line[0])
            vehicle_capacity = int(vehicle_number_line[1])

            customer_header_index = -1
            for i, line in enumerate(lines):
                 if "CUSTOMER" in line.upper() or (line.strip().startswith("CUST") and "XCOORD" in line.upper()):
                      customer_header_index = i
                      break

            if customer_header_index == -1 or customer_header_index + 1 >= len(lines):
                 raise ValueError("Customer data header not found.")

            customer_data_start_index = customer_header_index + 1
            while customer_data_start_index < len(lines) and not lines[customer_data_start_index].strip():
                 customer_data_start_index += 1

            if customer_data_start_index < len(lines) and "CUST" in lines[customer_data_start_index].upper() :
                 customer_data_start_index += 1


            for line_num, line in enumerate(lines[customer_data_start_index:], start=customer_data_start_index):
                parts = line.split()
                if len(parts) == 7:

                    x_coord = float(parts[1]) 
                    y_coord = float(parts[2])
                    demand = float(parts[3])  
                    ready_time = float(parts[4])
                    due_time = float(parts[5])
                    service_time = float(parts[6])

                    demands.append(demand)
                    time_windows.append((ready_time, due_time))
                    coords.append((x_coord, y_coord))
                    service_times.append(service_time)
                elif line.strip(): 
                     print(f"Warning: Skipping malformed line {line_num+1}: {line.strip()}")


            if not coords: 
                 raise ValueError("No valid customer data found.")

            coordinates_dict = {i: coord for i, coord in enumerate(coords)}

            return vehicle_number, vehicle_capacity, demands, time_windows, coords, service_times, coordinates_dict

        except (ValueError, IndexError, TypeError) as e:
             print(f"Error parsing Solomon data in {f_path}: {e}")
             return None 


    def calculate_distance_matrix(coords):
        n = len(coords)
        if n == 0:
            return []
        distance_matrix = [[0.0] * n for _ in range(n)]
        for i in range(n):
            for j in range(i + 1, n):

                if not all(isinstance(c, (int, float)) for c in coords[i] + coords[j]):
                     print(f"Warning: Invalid coordinates found at index {i} or {j}. Skipping distance calculation.")
                     dist = float('inf')
                else:
                     dist = math.sqrt((coords[i][0] - coords[j][0]) ** 2 + (coords[i][1] - coords[j][1]) ** 2)
                distance_matrix[i][j] = dist
                distance_matrix[j][i] = dist
        return distance_matrix

    print(f"Processing file: {file_path} with method: {method}, time limit: {time_limit}s")

    loaded_data = load_solomon_data(file_path)
    if loaded_data is None:
        print("Failed to load data.")
        return None, float('inf')

    vehicle_number, vehicle_capacity, demands, time_windows, coords, service_times, coordinates_dict = loaded_data

    if not coords or not demands or len(coords) != len(demands):
         print("Error: Inconsistent data loaded (coords/demands mismatch or empty).")
         return None, float('inf')
    num_customers_incl_depot = len(coords)
    print(f"Data loaded: {num_customers_incl_depot-1} customers, Capacity={vehicle_capacity}")


    distance_matrix = calculate_distance_matrix(coords)
    if not distance_matrix:
         print("Error: Failed to create distance matrix.")
         return None, float('inf')

    try:
        solver = CVRPTWSolver(
            distance_matrix=distance_matrix,
            vehicle_number=vehicle_number,
            vehicle_capacity=vehicle_capacity,
            demands=demands,
            time_windows=time_windows,
            service_times=service_times,
            initial_solution=initial_solution
        )

        solver.num_customers = num_customers_incl_depot - 1
        print("Solver initialized.")
        if initial_solution:
             print(f"Using provided initial solution with {len(initial_solution)} routes.")

    except NameError:
        print("Error: CVRPTWSolver class is not defined or imported.")
        return None, float('inf')
    except Exception as e:
        print(f"Error initializing CVRPTWSolver: {e}")
        return None, float('inf')


    print(f"Starting solver with method='{method}', time_limit={time_limit}, known_optimum={known_optimum}")
    try:
        best_solution, best_cost, solution_info = solver.solve(
            time_limit=time_limit,
            method=method,
            known_optimum=known_optimum
        )
    except Exception as e:
        print(f"Error during solver.solve execution: {e}")
        return getattr(solver, 'best_solution', None), getattr(solver, 'best_cost', float('inf'))


    print("-" * 20 + " Results " + "-" * 20)
    print(f"Best solution cost found: {best_cost:.4f}")

    if solution_info:
        print(f"Method used: {solution_info.get('method', 'N/A')}")
        print(f"Solution time: {solution_info.get('solution_time', 'N/A'):.2f}s")
        if 'num_routes' in solution_info:
             print(f"Number of routes: {solution_info.get('num_routes', 'N/A')}")

        final_routes_list = []
        if 'route_info' in solution_info and isinstance(solution_info['route_info'], list):
             for i, route_detail in enumerate(solution_info['route_info']):

                 if isinstance(route_detail, dict) and 'route' in route_detail:
                      route_str = route_detail.get('route', [])
                      print(f"\nRoute {i+1}: {route_str}")
                      print(f"  Distance: {route_detail.get('distance', 'N/A'):.2f}")
                      if isinstance(route_str, (list, tuple)) and any(c != 0 for c in route_str):
                           final_routes_list.append(route_str)
                 else:
                      print(f"\nRoute {i+1}: Invalid route detail format.")

             print("\nFinal non-empty routes list:")
             print(final_routes_list)
        else:
             print("\nRoute details ('route_info') not found or invalid in solution_info.")
             if best_solution and isinstance(best_solution, list):
                  final_routes_list = [route for route in best_solution if isinstance(route, list) and any(c != 0 for c in route)]
                  print("\nFinal non-empty routes list (from best_solution directly):")
                  print(final_routes_list)

    else:
         print("Solution info dictionary was not returned from solve.")
         if best_solution and isinstance(best_solution, list):
              final_routes_list = [route for route in best_solution if isinstance(route, list) and any(c != 0 for c in route)]
              print("\nFinal non-empty routes list (from best_solution directly):")
              print(final_routes_list)
         else:
              final_routes_list = None

    solver.visualize_solution(best_solution, coordinates_dict, pic_save_path, base_name)

    return final_routes_list, best_cost, solution_info

# test

## 1

In [ ]:
file_folder = ["test_data_100"]
base_path = "/home/wsl/CVRPTW/data"

original_excel_path = "/home/wsl/CVRPTW/test_data_initial copy.xlsx"
int_excel_path = "/home/wsl/CVRPTW/test_data_initial_int copy.xlsx"
excel_sources = {
    "original": original_excel_path,
    "integer": int_excel_path
}

methods_to_test = ["satl", "vnstl", "hybrid"]
runtimes_to_test = [5, 10, 30, 60, 100, 150, 210, 280, 360]
early_stop_count = 3

output_file_path = f"/home/wsl/CVRPTW/data/test_data_100/5/runtime_convergence_{'_'.join(file_folder)}_multi_initial_no_error_handling.csv"
pic_save_path_base = f"/home/wsl/CVRPTW/data/test_data_100/5/pics_runtime_convergence"

initial_solution_maps = {}
print("--- Pre-loading Initial Solutions ---")
for source_name, excel_path in excel_sources.items():
    print(f"Loading initial solutions from source '{source_name}': {os.path.basename(excel_path)}")
    df_current = pd.read_excel(excel_path)
    parsed_routes = {}
    if 'file_name' in df_current.columns and 'initial_route' in df_current.columns:
        for index, row in df_current.iterrows():
            fname = row['file_name']
            route_str = row['initial_route']
            if isinstance(route_str, str) and route_str.startswith('['):
                try:
                    parsed_routes[fname] = ast.literal_eval(route_str)
                except (ValueError, SyntaxError) as parse_error:
                    print(f"  Warning: Failed parsing route for {fname} in {source_name} source: {parse_error}. Entry skipped.")
    else:
        print(f"  Warning: Required columns ('file_name', 'initial_route') not found in {excel_path}")
    initial_solution_maps[source_name] = parsed_routes
    print(f"  Loaded {len(parsed_routes)} initial solutions from source '{source_name}'.")
print("--- Finished Pre-loading ---")

all_results_list = []
script_start_time = time.time()

for folder in file_folder:
    folder_path = os.path.join(base_path, folder)
    print(f"\n===== Processing Folder: {folder} =====")
    if not os.path.isdir(folder_path):
        print(f"Warning: Folder not found: {folder_path}. Skipping.")
        continue

    files_in_dir = os.listdir(folder_path)
    files = [f for f in files_in_dir
             if os.path.isfile(os.path.join(folder_path, f)) and f.lower().endswith('.txt')]
    print(f"Found {len(files)} '.txt' files to process.")
    if not files:
         print("No '.txt' files found in this folder.")
         continue

    for file_name in files:
        base_name = os.path.splitext(file_name)[0]
        target_file_link = os.path.join(folder_path, file_name)
        print(f"\n--- Processing File: {file_name} ({base_name}) ---")

        for source_name in excel_sources.keys():
            print(f"  -- Using Initial Solution Source: {source_name} --")

            initial_route_parsed = initial_solution_maps.get(source_name, {}).get(base_name)

            if initial_route_parsed is None:
                print(f"    Skipping source '{source_name}': No valid initial solution found/parsed for {base_name}.")
                continue
            print(f"    Initial solution loaded. Starting method tests...")

            for method in methods_to_test:
                print(f"    -- Running Method: {method} --")
                actual_solver_method = method

                best_cost_so_far_for_method = float('inf')
                last_n_best_costs = deque(maxlen=early_stop_count)

                for runtime in runtimes_to_test:
                    print(f"      -- Running Time Limit: {runtime}s --")
                    
                    run_routes, run_cost, run_info = produce_train_data(
                        file_path=target_file_link,
                        time_limit=runtime,
                        method=actual_solver_method,
                        known_optimum=None,
                        initial_solution=initial_route_parsed,
                        pic_save_path=None,
                        base_name=base_name
                    )

                    if run_info is None: run_info = {}
                    actual_solve_time = run_info.get("solution_time", runtime)
                    cost_at_this_runtime = run_cost 

                    if cost_at_this_runtime < best_cost_so_far_for_method:
                         best_cost_so_far_for_method = cost_at_this_runtime

                    print(f"        Cost at this runtime: {cost_at_this_runtime:.4f}, Best so far: {best_cost_so_far_for_method:.4f}, Actual Time: {actual_solve_time:.2f}s")

                    status = "Completed"
                    result_entry = {
                        "file_name": base_name,
                        "initial_solution_source": source_name,
                        "method": method,
                        "runtime_limit": runtime,
                        "cost_at_this_runtime": round(cost_at_this_runtime, 4) if cost_at_this_runtime != float('inf') else None,
                        "best_cost_so_far": round(best_cost_so_far_for_method, 4) if best_cost_so_far_for_method != float('inf') else None,
                        "actual_solve_time": round(actual_solve_time, 2),
                        "status": status, 
                    }
                    all_results_list.append(result_entry)

                    current_best_rounded = round(best_cost_so_far_for_method, 4)
                    last_n_best_costs.append(current_best_rounded)

                    if len(last_n_best_costs) == early_stop_count:
                         first_cost_in_deque = last_n_best_costs[0]

                         if all(cost == first_cost_in_deque for cost in last_n_best_costs):
                              print(f"        EARLY STOPPING: Best cost ({first_cost_in_deque:.4f}) hasn't improved for the last {early_stop_count} runtimes. Stopping runs for {method} with {source_name} initial solution.")

                              all_results_list[-1]["status"] = f"Completed (Early Stop Triggered after this run)"
                              break

print(f"\n--- Saving all results to {output_file_path} ---")
if all_results_list:
    results_df = pd.DataFrame(all_results_list)
    cols_order = [
        "file_name", "initial_solution_source", "method", "runtime_limit",
        "cost_at_this_runtime", "best_cost_so_far", "actual_solve_time", "status"
    ]
    cols_order = [col for col in cols_order if col in results_df.columns]
    results_df = results_df[cols_order]
    results_df.to_csv(output_file_path, index=False, encoding='utf-8')
    print(f"Successfully saved {len(all_results_list)} results.")
else:
    print("No results were generated to save (possibly due to early errors or no valid data).")

script_end_time = time.time()
print(f"\nTotal script execution time: {script_end_time - script_start_time:.2f} seconds.")